In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

import os
import sys
parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)

from newscrawler import NewsCrawler
from IPython.display import display
from IPython.core.debugger import set_trace

# 0. 초기화

In [8]:
crawler = NewsCrawler(storage='bigquery')

checking bigquery storage... done (26.80 seconds)
we have total 392113 articles (341337 downloaded, 50776 trashed)


# 1. URL을 모은다

In [9]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [10]:
collect_result = crawler.collect(); display(*collect_result)

100.00% (106.00 seconds): Collecting URLs... yahoo               

100.00% (0.13 seconds): Selecting URLs... cryptonews          

,collected,collected_unique,selected
all,25621,25553,526
yahoo,367,367,87
euronews,1004,1004,52
reuters,1081,1081,51
dailybeast,86,86,43
...,...,...,...
arstechnica,211,211,0
news24,1248,1244,0
thinkprogress,82,82,0
realclearpolitics,281,261,0


,url,pubs,actual_pub
0,https://politico.com/news/2019/11/05/mcconell-...,"politico, axios",politico
1,https://politico.com/magazine/story/2019/10/30...,"politico, realclearpolitics",politico
2,https://politico.com/magazine/story/2019/11/05...,"politico, realclearpolitics",politico
3,https://politico.com/news/2019/11/05/iowa-2020...,"politico, realclearpolitics",politico
4,https://politico.com/news/2019/11/05/7-things-...,"politico, realclearpolitics",politico
...,...,...,...
63,https://msnbc.com/mtp-daily/watch/indicted-giu...,"msnbc, nbcnews",msnbc
64,https://nbcnews.com/nightly-news/video/pete-bu...,"msnbc, nbcnews",nbcnews
65,https://nbcnews.com/pop-culture/viral/internet...,"msnbc, nbcnews",nbcnews
66,https://nbcnews.com/tech/security/facebook-say...,"msnbc, nbcnews",nbcnews


# 2. 해당 URL의 뉴스를 크롤링한다

In [11]:
crawl_result = crawler.crawl(); crawl_result

100.00% (388.49 seconds): Crawling... yahoo               



collect                                crawl              
                  collected collected_unique selected downloaded trashed total
all                   25621            25553      526        322     204   526
yahoo                   367              367       87         60      27    87
euronews               1004             1004       52          4      48    52
reuters                1081             1081       51         29      22    51
dailybeast               86               86       43         30      13    43
...                     ...              ...      ...        ...     ...   ...
arstechnica             211              211        0          0       0     0
news24                 1248             1244        0          0       0     0
thinkprogress            82               82        0          0       0     0
realclearpolitics       281              261        0          0       0     0
arynews                 122              122        0          0       0     0

[74 rows x 6 columns]

# 3. 뉴스를 저장한다

In [12]:
crawler.record()

Recording to bigquery storage... done (15.68 seconds)


# 4. 결과보고

In [13]:
crawler.report()

In [52]:
335490+144+138

335772

In [53]:
46904+153+158

47215

In [51]:
112/297

0.3771043771043771

In [66]:
from pathlib import Path
downloaded = [p.stem for p in Path('newsdata/downloaded').glob('**/*.json')]
trashed = [p.stem for p in Path('newsdata/trashed').glob('**/*.json')]

In [65]:
for path in list(Path('newsdata/downloaded').glob('**/*.json'))[:10000]:
    path.unlink()

In [53]:
from pathlib import Path
import json
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.header import Header

In [54]:
acc_fname = 'smtp_gmail.json'
acc = json.loads(Path(acc_fname).read_text()); acc

{'host': 'smtp.gmail.com',
 'from': 'gem763@gmail.com',
 'to': ['gem763@gmail.com', 'sejin.kang@hanwha.com'],
 'pw': 'kkangse1'}

In [55]:
msg = MIMEMultipart('alternative')
msg['Subject'] = Header('NEWS CRAWLING REPORT', 'utf-8')
msg['From'] = acc['from']
msg['To'] = ', '.join(acc['to'])
msg.attach(MIMEText(crawler.crawl_summary.to_html(), 'html'))

with smtplib.SMTP_SSL(acc['host']) as smtp:
    smtp.login(acc['from'], acc['pw'])
    smtp.send_message(msg)

In [1]:
import pandas as pd

In [13]:
str(pd.Timestamp.utcnow())[:19]

'2019-11-03 05:39:06'

In [15]:
import json
type(json.load(open('smtp_gmail.json')))

dict

In [32]:
def test():
    def now():
        return 1111

In [34]:
test().now()

AttributeError: 'NoneType' object has no attribute 'now'